# Purpose
This program generates a list of randomized perturbations of various parameters of a rocket's design, launch, and trajectory, then simulates a trajectory for each one and tabulates the perturbations and their final impact points. Impact points are in Cartesian coordinates, so columns are created for latitude, longitude, and height, and then the results are saved as a .csv file for later analysis by another program.

In order, the perturbations are as follows (in order):

| Column name |  Description | Units |
| ------- | --- | --- |
|'deg N' | Initial latitude offset | decimal degrees|
|'deg E' | Initial longitude offset| decimal degrees|
|'Launch Az'| Launcher azimuth| degrees |
|'Launch El'| Launcher elevation|  degrees|
|'Tip-Off'| Tip-off error is simulated | boolean|
|'Thrust Pitch'| Engine thrust vector misalignment, pitch | degrees|
|'Thrust Yaw'|Engine thrust vector misalignment, yaw | degrees|
|'mdot'| Mass flow rate| kg/s|
|'p_e'| Nozzle exit pressure| Pa |
|'mass'| Excess mass | positive number, kg |
|'drag'| Drag coefficient | dimensionless |
|'Wind'| Wind velocity is included| boolean|

I tried to pick reasonable estimates for standard deviations of these parameters. However, we still need engineers to select values reflective of what we can expect IRL.

In [1]:
%run Trajectory_Simulation.ipynb
import pandas as pd

/home/cory/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
amount = 100 # number of simulations per run

def mc_init(num_sims):
    results = []
    perturbation_list = []

    for i in range(num_sims):
        perturbation = []
        perturbation.append(np.random.normal(0, 0.008333 / 3))
        perturbation.append(np.random.normal(0, 0.008333 / 3))
        perturbation.append(np.random.normal(0, 0.333333))
        perturbation.append(np.random.normal(0, 0.333333))
        perturbation.append(np.random.rand() < 0.25)
        perturbation.append(np.random.normal(0, 0.0333333))
        perturbation.append(np.random.normal(0, 0.0333333))
        perturbation.append(np.random.normal(0, 0.0333333 / 2))
        perturbation.append(np.random.normal(0, 0.0333333 / 2))
        perturbation.append(np.random.exponential(2.5))
        perturbation.append(np.random.normal(0, 0.0333333/2))
        perturbation.append(np.random.rand() < 0.25)
        
        perturbation_list.append(perturbation)
        
    print('random perturbations obtained\n')
    
    for i, perturbation in enumerate(perturbation_list):
        if i % 10 == 0: print('iterations:', i)
        sim = trajectory(144.589, 2.871, 74610.004, throttle_window, min_throttle, rcs_mdot, rcs_p_e, rcs_p_ch, 
                        ballast, root, tip, sweep, span, thickness, airfrm_in_rad,
                          OF, p_ch, T_ch, ke, MM,
                          perturbation, 
                          0.05, True, 0.045, True, True)
        x, y, z = sim.raw_states[-1][0][1]
        perturbation.append(x)
        perturbation.append(y)
        perturbation.append(z)
        
        results.append(perturbation)
    print('done simulations!\n')
    
    return results

In [5]:
def lat_long(df):
    array = df.loc[:, ['x', 'y', 'z']]
    X = [x for x in array['x']]
    Y = [y for y in array['y']]
    Z = [z for z in array['z']]
    coords = [Environment(None, 1).ECEF_to_geodetic([X[i], Y[i], Z[i]])
                                   for i in range(len(array))]
    df['lat'] = [coord[0] for coord in coords]
    df['long'] = [coord[1] for coord in coords]
    df['height'] = [coord[2] for coord in coords]
    
def handle_data(results):
    # landing coordinates
    results_df = pd.DataFrame.from_records(results,
                                           columns=['deg N', 'deg E', 'Launch Az', 'Launch El', 'Tip-Off',
                                                    'Thrust Pitch', 'Thrust Yaw', 'mdot', 'p_e',
                                                    'mass', 'drag', 'Wind',
                                                    'x', 'y', 'z'])
    lat_long(results_df)
    print(results_df.describe())
    return results_df

In [6]:
results = mc_init(num_sims=amount)
sim_data = handle_data(results)
sim_data.to_csv(path_or_buf='./dispersion_sample_data/sim_data1.csv')

            deg N       deg E   Launch Az   Launch El  Thrust Pitch  \
count  100.000000  100.000000  100.000000  100.000000    100.000000   
mean    -0.000124   -0.000202    0.034414    0.031812      0.000810   
std      0.002724    0.002763    0.317190    0.360150      0.034585   
min     -0.007077   -0.006319   -0.608312   -1.173812     -0.102277   
25%     -0.001963   -0.001889   -0.161482   -0.186783     -0.027559   
50%     -0.000239   -0.000255   -0.005540    0.042110     -0.000942   
75%      0.001668    0.001402    0.244786    0.215058      0.026047   
max      0.008254    0.005785    0.961040    0.857204      0.095089   

       Thrust Yaw        mdot         p_e        mass        drag  \
count  100.000000  100.000000  100.000000  100.000000  100.000000   
mean    -0.002749   -0.001024    0.000529    2.545613   -0.000744   
std      0.031692    0.018734    0.016645    2.357153    0.016035   
min     -0.089852   -0.049756   -0.040095    0.075692   -0.039751   
25%     -0.0192

In [7]:
results = mc_init(num_sims=amount)
sim_data = handle_data(results)
sim_data.to_csv(path_or_buf='./dispersion_sample_data/sim_data2.csv')

random perturbations obtained

iterations: 0
iterations: 10
iterations: 20
iterations: 30
iterations: 40
iterations: 50
iterations: 60
iterations: 70
iterations: 80
iterations: 90
done simulations!

            deg N       deg E   Launch Az   Launch El  Thrust Pitch  \
count  100.000000  100.000000  100.000000  100.000000    100.000000   
mean     0.000296   -0.000466   -0.053086    0.021562     -0.000514   
std      0.002999    0.002565    0.302299    0.357251      0.034457   
min     -0.006315   -0.006563   -0.671837   -0.889543     -0.080585   
25%     -0.002165   -0.002027   -0.270355   -0.254540     -0.024972   
50%      0.000046   -0.000489   -0.090759    0.003695      0.002786   
75%      0.002716    0.001322    0.201720    0.288334      0.022449   
max      0.009081    0.005910    0.516824    0.891808      0.100240   

       Thrust Yaw        mdot         p_e        mass        drag  \
count  100.000000  100.000000  100.000000  100.000000  100.000000   
mean    -0.000557    0.

In [8]:
results = mc_init(num_sims=amount)
sim_data = handle_data(results)
sim_data.to_csv(path_or_buf='./dispersion_sample_data/sim_data3.csv')

random perturbations obtained

iterations: 0
iterations: 10
iterations: 20
iterations: 30
iterations: 40
iterations: 50
iterations: 60
iterations: 70
iterations: 80
iterations: 90
done simulations!

            deg N       deg E   Launch Az   Launch El  Thrust Pitch  \
count  100.000000  100.000000  100.000000  100.000000    100.000000   
mean     0.000105   -0.000035    0.034149    0.034969     -0.006497   
std      0.002618    0.002742    0.301620    0.339893      0.033989   
min     -0.005571   -0.006930   -0.681254   -0.761620     -0.079309   
25%     -0.001884   -0.001718   -0.206955   -0.175747     -0.029068   
50%     -0.000044   -0.000037    0.044794   -0.010327     -0.004641   
75%      0.002201    0.001617    0.282479    0.290110      0.013340   
max      0.005915    0.006820    0.655535    0.914575      0.082844   

       Thrust Yaw        mdot         p_e        mass        drag  \
count  100.000000  100.000000  100.000000  100.000000  100.000000   
mean    -0.003996   -0.

In [9]:
results = mc_init(num_sims=amount)
sim_data = handle_data(results)
sim_data.to_csv(path_or_buf='./dispersion_sample_data/sim_data4.csv')

random perturbations obtained

iterations: 0
iterations: 10
iterations: 20
iterations: 30
iterations: 40
iterations: 50
iterations: 60
iterations: 70
iterations: 80
iterations: 90
done simulations!

            deg N       deg E   Launch Az   Launch El  Thrust Pitch  \
count  100.000000  100.000000  100.000000  100.000000    100.000000   
mean    -0.000038   -0.000310    0.072363   -0.011925     -0.004744   
std      0.002913    0.003065    0.337922    0.357207      0.031978   
min     -0.007139   -0.007930   -0.919341   -0.715641     -0.111476   
25%     -0.001985   -0.002666   -0.188051   -0.301481     -0.023906   
50%     -0.000042   -0.000425    0.021493   -0.082138     -0.000822   
75%      0.002275    0.001931    0.324132    0.211557      0.015244   
max      0.006817    0.007342    0.956446    0.885137      0.066249   

       Thrust Yaw        mdot         p_e        mass        drag  \
count  100.000000  100.000000  100.000000  100.000000  100.000000   
mean     0.003633   -0.

In [10]:
results = mc_init(num_sims=amount)
sim_data = handle_data(results)
sim_data.to_csv(path_or_buf='./dispersion_sample_data/sim_data5.csv')

random perturbations obtained

iterations: 0


KeyboardInterrupt: 